## Ejemplo 6: Automatizando peticiones

### 1. Objetivos:
    - Usar todo lo que aprendimos para automatizar peticiones al API
    - Guardar nuestros resultados en un archivo tipo .csv
 
---
    
### 2. Desarrollo:

Veamos cómo usar todo lo que aprendimos para automatizar el proceso de realizar múltiples peticiones a la API, reunirlas en un `DataFrame` y guardarlo en un .csv:

In [1]:
import pandas as pd
import requests

In [2]:
url = 'https://api.nasa.gov/neo/rest/v1/neo/browse/'
parametros = {
    'api_key': 'y8zWjcsWb7gpD8KNst9khzgRg2reNpMZiKa4rLeW'
}

Obtener 1 página de datos del API.

In [3]:
resultados = requests.get(url, params=parametros)
if resultados.status_code == 200:
    datos = resultados.json() # regresa un diccionario
    objetos = datos["near_earth_objects"] # se obtienen 20 objetos por página
    url = datos["links"]["next"]
    parametros = {}
else:
    print("Error en la petición de requests")

Obtener 10 páginas de datos del API.

In [4]:
url = 'https://api.nasa.gov/neo/rest/v1/neo/browse/'
parametros = {
    'api_key': 'y8zWjcsWb7gpD8KNst9khzgRg2reNpMZiKa4rLeW'
}
paginas_objetos = []
for _ in range(10):
    resultados = requests.get(url, params=parametros)
    if resultados.status_code == 200:
        datos = resultados.json() # regresa un diccionario
        objetos = datos["near_earth_objects"] # se obtienen 20 objetos por página
        paginas_objetos.append(objetos)
        
        url = datos["links"]["next"]
        parametros = {}
        
        print(datos["links"]["self"])
    else:
        print("Error en la petición de requests")

http://www.neowsapp.com/rest/v1/neo/browse?page=0&size=20&api_key=y8zWjcsWb7gpD8KNst9khzgRg2reNpMZiKa4rLeW
http://www.neowsapp.com/rest/v1/neo/browse?page=1&size=20&api_key=y8zWjcsWb7gpD8KNst9khzgRg2reNpMZiKa4rLeW
http://www.neowsapp.com/rest/v1/neo/browse?page=2&size=20&api_key=y8zWjcsWb7gpD8KNst9khzgRg2reNpMZiKa4rLeW
http://www.neowsapp.com/rest/v1/neo/browse?page=3&size=20&api_key=y8zWjcsWb7gpD8KNst9khzgRg2reNpMZiKa4rLeW
http://www.neowsapp.com/rest/v1/neo/browse?page=4&size=20&api_key=y8zWjcsWb7gpD8KNst9khzgRg2reNpMZiKa4rLeW
http://www.neowsapp.com/rest/v1/neo/browse?page=5&size=20&api_key=y8zWjcsWb7gpD8KNst9khzgRg2reNpMZiKa4rLeW
http://www.neowsapp.com/rest/v1/neo/browse?page=6&size=20&api_key=y8zWjcsWb7gpD8KNst9khzgRg2reNpMZiKa4rLeW
http://www.neowsapp.com/rest/v1/neo/browse?page=7&size=20&api_key=y8zWjcsWb7gpD8KNst9khzgRg2reNpMZiKa4rLeW
http://www.neowsapp.com/rest/v1/neo/browse?page=8&size=20&api_key=y8zWjcsWb7gpD8KNst9khzgRg2reNpMZiKa4rLeW
http://www.neowsapp.com/rest/v1/neo/b

Valiando la cantidad de diccionarios obtenidos:

In [5]:
len(paginas_objetos)

10

Validando el tipo de dato del elemento 0 de la lista:

In [6]:
type(paginas_objetos[0])

list

In [7]:
len(paginas_objetos[0])

20

In [8]:
type(paginas_objetos[0][0])

dict

Normalizar y crea los dataframes usando `map` o listas de compresión:

In [9]:
pd.json_normalize(paginas_objetos[0]).head(3)

,id,neo_reference_id,name,name_limited,designation,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,...,orbital_data.perihelion_distance,orbital_data.perihelion_argument,orbital_data.aphelion_distance,orbital_data.perihelion_time,orbital_data.mean_anomaly,orbital_data.mean_motion,orbital_data.equinox,orbital_data.orbit_class.orbit_class_type,orbital_data.orbit_class.orbit_class_description,orbital_data.orbit_class.orbit_class_range
0,2000433,2000433,433 Eros (A898 PA),Eros,433,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000433,10.43,False,"[{'close_approach_date': '1900-12-27', 'close_...",False,...,1.133475820116491,178.8971619902451,1.78307038061344,2459802.569605474861,246.9041236220357,.5596877180622503,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
1,2000719,2000719,719 Albert (A911 TB),Albert,719,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000719,15.51,False,"[{'close_approach_date': '1909-08-21', 'close_...",False,...,1.194898892415675,156.2275765643465,4.080111812929026,2459956.011943464095,278.1971857577304,.2300986387269753,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
2,2000887,2000887,887 Alinda (A918 AA),Alinda,887,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000887,13.87,False,"[{'close_approach_date': '1910-01-04', 'close_...",False,...,1.062256387810841,350.4932417946612,3.884059710445925,2459258.734847818714,86.60720939790032,.2534114693822276,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU


In [10]:
dfs = [pd.json_normalize(pagina) for pagina in paginas_objetos]

Valida el tipo de dato del elemento 0:

In [11]:
dfs[0].head(3)

,id,neo_reference_id,name,name_limited,designation,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,...,orbital_data.perihelion_distance,orbital_data.perihelion_argument,orbital_data.aphelion_distance,orbital_data.perihelion_time,orbital_data.mean_anomaly,orbital_data.mean_motion,orbital_data.equinox,orbital_data.orbit_class.orbit_class_type,orbital_data.orbit_class.orbit_class_description,orbital_data.orbit_class.orbit_class_range
0,2000433,2000433,433 Eros (A898 PA),Eros,433,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000433,10.43,False,"[{'close_approach_date': '1900-12-27', 'close_...",False,...,1.133475820116491,178.8971619902451,1.78307038061344,2459802.569605474861,246.9041236220357,.5596877180622503,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
1,2000719,2000719,719 Albert (A911 TB),Albert,719,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000719,15.51,False,"[{'close_approach_date': '1909-08-21', 'close_...",False,...,1.194898892415675,156.2275765643465,4.080111812929026,2459956.011943464095,278.1971857577304,.2300986387269753,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
2,2000887,2000887,887 Alinda (A918 AA),Alinda,887,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000887,13.87,False,"[{'close_approach_date': '1910-01-04', 'close_...",False,...,1.062256387810841,350.4932417946612,3.884059710445925,2459258.734847818714,86.60720939790032,.2534114693822276,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU


Explora el inicio del dataframe del elemento 0:

In [12]:
dfs[-1].head(3)

,id,neo_reference_id,name,designation,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,...,orbital_data.perihelion_argument,orbital_data.aphelion_distance,orbital_data.perihelion_time,orbital_data.mean_anomaly,orbital_data.mean_motion,orbital_data.equinox,orbital_data.orbit_class.orbit_class_type,orbital_data.orbit_class.orbit_class_description,orbital_data.orbit_class.orbit_class_range,name_limited
0,2014402,2014402,14402 (1991 DB),14402,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2014402,18.64,False,"[{'close_approach_date': '1937-06-16', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/2014402?ap...,...,51.35129373572945,2.404058973669982,2459858.336632440265,246.8133964768987,.4389857346951037,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU,NaN
1,2014827,2014827,14827 Hypnos (1986 JK),14827,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2014827,18.30,True,"[{'close_approach_date': '1916-09-11', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/2014827?ap...,...,238.1739128996268,4.735613546512233,2458744.323831768079,176.0623259256648,.2056379661784432,J2000,APO,Near-Earth asteroid orbits which cross the Ear...,a (semi-major axis) > 1.0 AU; q (perihelion) <...,Hypnos
2,2015745,2015745,15745 Yuliya (1991 PM5),15745,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2015745,17.32,False,"[{'close_approach_date': '1903-07-08', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/2015745?ap...,...,140.6009004747771,2.158364705377555,2459949.750320950117,207.3698892193037,.4370221059939874,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU,Yuliya


Concatena los dataframes en un sólo dataframe y reinicia índices (por alguna buena razón hemos creado una lista de dataframes ¿cierto?):

In [13]:
df = pd.concat(dfs)
df_1 = df.reset_index(drop=True)
df_1

,id,neo_reference_id,name,name_limited,designation,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,...,orbital_data.perihelion_distance,orbital_data.perihelion_argument,orbital_data.aphelion_distance,orbital_data.perihelion_time,orbital_data.mean_anomaly,orbital_data.mean_motion,orbital_data.equinox,orbital_data.orbit_class.orbit_class_type,orbital_data.orbit_class.orbit_class_description,orbital_data.orbit_class.orbit_class_range
0,2000433,2000433,433 Eros (A898 PA),Eros,433,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000433,10.43,False,"[{'close_approach_date': '1900-12-27', 'close_...",False,...,1.133475820116491,178.8971619902451,1.78307038061344,2459802.569605474861,246.9041236220357,.5596877180622503,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
1,2000719,2000719,719 Albert (A911 TB),Albert,719,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000719,15.51,False,"[{'close_approach_date': '1909-08-21', 'close_...",False,...,1.194898892415675,156.2275765643465,4.080111812929026,2459956.011943464095,278.1971857577304,.2300986387269753,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
2,2000887,2000887,887 Alinda (A918 AA),Alinda,887,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000887,13.87,False,"[{'close_approach_date': '1910-01-04', 'close_...",False,...,1.062256387810841,350.4932417946612,3.884059710445925,2459258.734847818714,86.60720939790032,.2534114693822276,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
3,2001036,2001036,1036 Ganymed (A924 UB),Ganymed,1036,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2001036,9.25,False,"[{'close_approach_date': '1910-02-25', 'close_...",False,...,1.244627432512162,132.4281002671606,4.087071305845932,2458979.339225097543,140.6548420252212,.2264387059007529,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
4,2001221,2001221,1221 Amor (1932 EA1),Amor,1221,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2001221,17.39,False,"[{'close_approach_date': '1900-03-08', 'close_...",False,...,1.08246658685257,26.64392069638918,2.754995802947215,2459867.343755315685,261.0445055510584,.3708368379536327,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2017511,2017511,17511 (1992 QN),NaN,17511,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2017511,17.21,False,"[{'close_approach_date': '1900-12-11', 'close_...",False,...,.7628281226328586,202.3610757900837,1.61707495514811,2459638.216060239377,331.3624090068763,.7592943380450243,J2000,APO,Near-Earth asteroid orbits which cross the Ear...,a (semi-major axis) > 1.0 AU; q (perihelion) <...
196,2018106,2018106,18106 Blume (2000 NX3),Blume,18106,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2018106,17.70,False,"[{'close_approach_date': '1904-08-28', 'close_...",False,...,1.193207942724581,234.9477519388817,3.696580287030257,2460164.248215992648,214.6553094085096,.2578184488540994,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
197,2018109,2018109,18109 (2000 NG11),NaN,18109,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2018109,17.08,False,"[{'close_approach_date': '1902-10-30', 'close_...",False,...,1.189441890868818,319.2489937646262,2.572955874147814,2459350.606817411927,95.45669944686918,.3819900105247013,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
198,2018172,2018172,18172 (2000 QL7),NaN,18172,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2018172,15.61,False,"[{'close_approach_date': '1913-11-01', 'close_...",False,...,1.184930012194684,100.7343072984311,3.66298690061217,2460208.249589382581,201.2765884333697,.2611658063444833,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU


Comprobemos la cantidad de objetos en el dataframe final, 10 páginas con 20 objetos cada una:

In [14]:
df_1.shape

(200, 44)

Guarda tu dataframe en el archivo `objetos_tierra_nasa.csv`:

In [15]:
df_1.to_csv("objetos_tierra_nasa.csv")

---
---

## Reto 6: Automatizando peticiones

### 1. Objetivos:
    - Automatizar la petición de datos a la API de la NASA por fechas.
 
---
    
### 2. Desarrollo:

In [17]:
import datetime

In [18]:
url = 'https://api.nasa.gov/neo/rest/v1/neo/browse/'
api_key ='y8zWjcsWb7gpD8KNst9khzgRg2reNpMZiKa4rLeW'

start_date = '1995-01-01'
end_date = '1995-02-28' 

parametros = {
    'api_key': api_key,
    '': start_date,
    '': end_date
}

### a) Automatizando peticiones

En el primer Reto realizaste una petición a la API de la NASA para pedir 5 datos de la hoja #100. Ahora, vamos a automatizar el proceso de realizar peticiones a la API para obtener una cantidad bastante mayor de datos.

Vamos a obtener ahora los datos por fechas, así que tendrás que ir a la página de la [api de la NASA](https://api.nasa.gov/), para revisar cómo es posible realizar peticiones por fecha. Queremos obtener los **dos primeros meses (Enero y Febrero) del año 1995**. Observa que la API sólo permite peticiones por fecha en rangos de 7 días.

Tu reto tiene los siguientes pasos:

1. Revisa la documentación de la API de la NASA para entender cómo realizar peticiones por fecha.
2. Realiza una primera petición de prueba para entender el formato de los datos que obtienes de regreso (cómo extraemos los datos que necesitamos y qué estructura tienen).
2. Escribe el código necesario para automatizar las peticiones a la API y obtener los meses de Enero y Febrero del año 1995.
3. Almacena los datos de cada petición y luego usa esos datos para crear `DataFrames`.
4. Concatena verticalmente tus `DataFrames` para obtener un `DataFrame` final que contenga todos los datos de tus peticiones. Cada fila tiene que corresponder a un objeto espacial.
5. Guarda tu `DataFrame` con el nombre de `near_earth_objects-january_february_1995-raw.csv`.

> **Nota**: En este momento no te preocupes por explorar o limpiar tu dataset. Eso lo haremos en la siguiente sesión. Lo que sí tienes que asegurarte es de normalizar tus datos antes de convertirlos en `DataFrame`.

¡Mucha suerte!

In [39]:
def traer_datos_neo(url, parametros):
    """ Obtiene los datos de los obejetos cercanos a la tierra, de acuerdo a la url y los parámetros dados"""
    resultados = requests.get(url, params = parametros)
    if resultados.status_code == 200:
        datos = resultados.json() # regresa un diccionario
        objetos = datos["near_earth_objects"] # se obtienen 20 objetos por página
        return objetos 
    else:
        print("Error en la petición de requests")
        return None

In [ ]:
datos = traer_datos_neo(url, parametros)

In [ ]:
pd.json_normalize(datos)

In [40]:
url = 'https://api.nasa.gov/neo/rest/v1/neo/browse/'
api_key ='y8zWjcsWb7gpD8KNst9khzgRg2reNpMZiKa4rLeW'

start_date = datetime.datetime.strptime('1995-01-01','%Y-%m-%d') 
end_date = datetime.datetime.strptime('1995-02-28','%Y-%m-%d') 

parametros = {
    'api_key': api_key,
    '': start_date,
    '': end_date
}
start_date

datetime.datetime(1995, 1, 1, 0, 0)

In [41]:
lista_datos = []
while start_date <= ( end_date ):
    start_date_p = start_date.strftime('%Y-%m-%d')
    
    if start_date + datetime.timedelta(days=7) > end_date :
        end_date_p = end_date.strftime('%Y-%m-%d')
    else:
        end_date_p = (start_date + datetime.timedelta(days=7)).strftime('%Y-%m-%d')
    
    parametros = {
        'api_key': api_key,
        'start_date': start_date_p,
        'end_date': end_date_p,
    }
    print(parametros)
    
    lista_datos.append(traer_datos_neo(url, parametros))
    
    start_date += datetime.timedelta(days=7)

{'api_key': 'y8zWjcsWb7gpD8KNst9khzgRg2reNpMZiKa4rLeW', 'start_date': '1995-01-01', 'end_date': '1995-01-08'}
{'api_key': 'y8zWjcsWb7gpD8KNst9khzgRg2reNpMZiKa4rLeW', 'start_date': '1995-01-08', 'end_date': '1995-01-15'}
{'api_key': 'y8zWjcsWb7gpD8KNst9khzgRg2reNpMZiKa4rLeW', 'start_date': '1995-01-15', 'end_date': '1995-01-22'}
{'api_key': 'y8zWjcsWb7gpD8KNst9khzgRg2reNpMZiKa4rLeW', 'start_date': '1995-01-22', 'end_date': '1995-01-29'}
{'api_key': 'y8zWjcsWb7gpD8KNst9khzgRg2reNpMZiKa4rLeW', 'start_date': '1995-01-29', 'end_date': '1995-02-05'}
{'api_key': 'y8zWjcsWb7gpD8KNst9khzgRg2reNpMZiKa4rLeW', 'start_date': '1995-02-05', 'end_date': '1995-02-12'}
{'api_key': 'y8zWjcsWb7gpD8KNst9khzgRg2reNpMZiKa4rLeW', 'start_date': '1995-02-12', 'end_date': '1995-02-19'}
{'api_key': 'y8zWjcsWb7gpD8KNst9khzgRg2reNpMZiKa4rLeW', 'start_date': '1995-02-19', 'end_date': '1995-02-26'}
{'api_key': 'y8zWjcsWb7gpD8KNst9khzgRg2reNpMZiKa4rLeW', 'start_date': '1995-02-26', 'end_date': '1995-02-28'}


In [43]:
len(lista_datos)

9

In [44]:
lista_datos[0]

[{'links': {'self': 'http://www.neowsapp.com/rest/v1/neo/2000433?api_key=y8zWjcsWb7gpD8KNst9khzgRg2reNpMZiKa4rLeW'},
  'id': '2000433',
  'neo_reference_id': '2000433',
  'name': '433 Eros (A898 PA)',
  'name_limited': 'Eros',
  'designation': '433',
  'nasa_jpl_url': 'http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000433',
  'absolute_magnitude_h': 10.43,
  'estimated_diameter': {'kilometers': {'estimated_diameter_min': 21.8049440483,
    'estimated_diameter_max': 48.7573371375},
   'meters': {'estimated_diameter_min': 21804.9440482866,
    'estimated_diameter_max': 48757.3371375482},
   'miles': {'estimated_diameter_min': 13.5489598882,
    'estimated_diameter_max': 30.2963953345},
   'feet': {'estimated_diameter_min': 71538.5326313805,
    'estimated_diameter_max': 159965.0219743538}},
  'is_potentially_hazardous_asteroid': False,
  'close_approach_data': [{'close_approach_date': '1900-12-27',
    'close_approach_date_full': '1900-Dec-27 01:30',
    'epoch_date_close_approach': -21778794000

In [46]:
pd.json_normalize(lista_datos[1])

,id,neo_reference_id,name,name_limited,designation,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,...,orbital_data.perihelion_distance,orbital_data.perihelion_argument,orbital_data.aphelion_distance,orbital_data.perihelion_time,orbital_data.mean_anomaly,orbital_data.mean_motion,orbital_data.equinox,orbital_data.orbit_class.orbit_class_type,orbital_data.orbit_class.orbit_class_description,orbital_data.orbit_class.orbit_class_range
0,2000433,2000433,433 Eros (A898 PA),Eros,433,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000433,10.43,False,"[{'close_approach_date': '1900-12-27', 'close_...",False,...,1.133475820116491,178.8971619902451,1.78307038061344,2459802.569605474861,246.9041236220357,.5596877180622503,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
1,2000719,2000719,719 Albert (A911 TB),Albert,719,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000719,15.51,False,"[{'close_approach_date': '1909-08-21', 'close_...",False,...,1.194898892415675,156.2275765643465,4.080111812929026,2459956.011943464095,278.1971857577304,.2300986387269753,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
2,2000887,2000887,887 Alinda (A918 AA),Alinda,887,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000887,13.87,False,"[{'close_approach_date': '1910-01-04', 'close_...",False,...,1.062256387810841,350.4932417946612,3.884059710445925,2459258.734847818714,86.60720939790032,.2534114693822276,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
3,2001036,2001036,1036 Ganymed (A924 UB),Ganymed,1036,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2001036,9.25,False,"[{'close_approach_date': '1910-02-25', 'close_...",False,...,1.244627432512162,132.4281002671606,4.087071305845932,2458979.339225097543,140.6548420252212,.2264387059007529,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
4,2001221,2001221,1221 Amor (1932 EA1),Amor,1221,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2001221,17.39,False,"[{'close_approach_date': '1900-03-08', 'close_...",False,...,1.08246658685257,26.64392069638918,2.754995802947215,2459867.343755315685,261.0445055510584,.3708368379536327,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
5,2001566,2001566,1566 Icarus (1949 MA),Icarus,1566,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2001566,16.35,True,"[{'close_approach_date': '1902-06-11', 'close_...",False,...,.1865708998565535,31.43750761745641,1.969614892641781,2459600.124875274151,.3302900481594904,.8804806119135474,J2000,APO,Near-Earth asteroid orbits which cross the Ear...,a (semi-major axis) > 1.0 AU; q (perihelion) <...
6,2001580,2001580,1580 Betulia (1950 KA),Betulia,1580,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2001580,14.69,False,"[{'close_approach_date': '1911-05-07', 'close_...",False,...,1.126868095165017,159.5134530240992,3.267708226754692,2459545.547142946129,16.62891169416756,.3026032236661684,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
7,2001620,2001620,1620 Geographos (1951 RA),Geographos,1620,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2001620,15.29,True,"[{'close_approach_date': '1901-08-23', 'close_...",False,...,.8279319618844114,276.9463790166408,1.663446614878227,2459684.442498765332,300.4925456829043,.7089073495828783,J2000,APO,Near-Earth asteroid orbits which cross the Ear...,a (semi-major axis) > 1.0 AU; q (perihelion) <...
8,2001627,2001627,1627 Ivar (1929 SH),Ivar,1627,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2001627,12.67,False,"[{'close_approach_date': '1901-07-11', 'close_...",False,...,1.124043931637646,167.8035837640704,2.602502927777894,2459267.535393223541,129.0289785125129,.3875155974134465,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
9,2001685,2001685,1685 Toro (1948 OA),Toro,1685,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=

In [47]:
pd.json_normalize(lista_datos[0])

,id,neo_reference_id,name,name_limited,designation,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,...,orbital_data.perihelion_distance,orbital_data.perihelion_argument,orbital_data.aphelion_distance,orbital_data.perihelion_time,orbital_data.mean_anomaly,orbital_data.mean_motion,orbital_data.equinox,orbital_data.orbit_class.orbit_class_type,orbital_data.orbit_class.orbit_class_description,orbital_data.orbit_class.orbit_class_range
0,2000433,2000433,433 Eros (A898 PA),Eros,433,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000433,10.43,False,"[{'close_approach_date': '1900-12-27', 'close_...",False,...,1.133475820116491,178.8971619902451,1.78307038061344,2459802.569605474861,246.9041236220357,.5596877180622503,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
1,2000719,2000719,719 Albert (A911 TB),Albert,719,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000719,15.51,False,"[{'close_approach_date': '1909-08-21', 'close_...",False,...,1.194898892415675,156.2275765643465,4.080111812929026,2459956.011943464095,278.1971857577304,.2300986387269753,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
2,2000887,2000887,887 Alinda (A918 AA),Alinda,887,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000887,13.87,False,"[{'close_approach_date': '1910-01-04', 'close_...",False,...,1.062256387810841,350.4932417946612,3.884059710445925,2459258.734847818714,86.60720939790032,.2534114693822276,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
3,2001036,2001036,1036 Ganymed (A924 UB),Ganymed,1036,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2001036,9.25,False,"[{'close_approach_date': '1910-02-25', 'close_...",False,...,1.244627432512162,132.4281002671606,4.087071305845932,2458979.339225097543,140.6548420252212,.2264387059007529,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
4,2001221,2001221,1221 Amor (1932 EA1),Amor,1221,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2001221,17.39,False,"[{'close_approach_date': '1900-03-08', 'close_...",False,...,1.08246658685257,26.64392069638918,2.754995802947215,2459867.343755315685,261.0445055510584,.3708368379536327,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
5,2001566,2001566,1566 Icarus (1949 MA),Icarus,1566,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2001566,16.35,True,"[{'close_approach_date': '1902-06-11', 'close_...",False,...,.1865708998565535,31.43750761745641,1.969614892641781,2459600.124875274151,.3302900481594904,.8804806119135474,J2000,APO,Near-Earth asteroid orbits which cross the Ear...,a (semi-major axis) > 1.0 AU; q (perihelion) <...
6,2001580,2001580,1580 Betulia (1950 KA),Betulia,1580,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2001580,14.69,False,"[{'close_approach_date': '1911-05-07', 'close_...",False,...,1.126868095165017,159.5134530240992,3.267708226754692,2459545.547142946129,16.62891169416756,.3026032236661684,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
7,2001620,2001620,1620 Geographos (1951 RA),Geographos,1620,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2001620,15.29,True,"[{'close_approach_date': '1901-08-23', 'close_...",False,...,.8279319618844114,276.9463790166408,1.663446614878227,2459684.442498765332,300.4925456829043,.7089073495828783,J2000,APO,Near-Earth asteroid orbits which cross the Ear...,a (semi-major axis) > 1.0 AU; q (perihelion) <...
8,2001627,2001627,1627 Ivar (1929 SH),Ivar,1627,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2001627,12.67,False,"[{'close_approach_date': '1901-07-11', 'close_...",False,...,1.124043931637646,167.8035837640704,2.602502927777894,2459267.535393223541,129.0289785125129,.3875155974134465,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
9,2001685,2001685,1685 Toro (1948 OA),Toro,1685,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=

In [48]:
df_normalizados = [pd.json_normalize(dato) for dato in lista_datos]

In [52]:
for i in range(len(df_normalizados)):
    print(i) 

0
1
2
3
4
5
6
7
8


In [55]:
df_concatenado=pd.concat(df_normalizados, keys=range(len(df_normalizados)) )
df_concatenado

id neo_reference_id                         name  name_limited  \
0 0   2000433          2000433           433 Eros (A898 PA)          Eros   
  1   2000719          2000719         719 Albert (A911 TB)        Albert   
  2   2000887          2000887         887 Alinda (A918 AA)        Alinda   
  3   2001036          2001036       1036 Ganymed (A924 UB)       Ganymed   
  4   2001221          2001221         1221 Amor (1932 EA1)          Amor   
...       ...              ...                          ...           ...   
8 15  2001915          2001915  1915 Quetzalcoatl (1953 EA)  Quetzalcoatl   
  16  2001916          2001916        1916 Boreas (1953 RA)        Boreas   
  17  2001917          2001917          1917 Cuyo (1968 AA)          Cuyo   
  18  2001943          2001943       1943 Anteros (1973 EC)       Anteros   
  19  2001980          2001980  1980 Tezcatlipoca (1950 LA)  Tezcatlipoca   

     designation                                   nasa_jpl_url  \
0 0          433  http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000433   
  1          719  http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000719   
  2          887  http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000887   
  3         1036  http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2001036   
  4         1221  http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2001221   
...          ...                                            ...   
8 15        1915  http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2001915   
  16        1916  http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2001916   
  17        1917  http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2001917   
  18        1943  http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2001943   
  19        1980  http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2001980   

      absolute_magnitude_h  is_potentially_hazardous_asteroid  \
0 0                  10.43                              False   
  1                  15.51                              False   
  2                  13.87                              False   
  3                   9.25                              False   
  4                  17.39                              False   
...                    ...                                ...   
8 15                 18.97                              False   
  16                 14.88                              False   
  17                 14.32                              False   
  18                 15.65                              False   
  19                 13.80                              False   

                                    close_approach_data  is_sentry_object  \
0 0   [{'close_approach_date': '1900-12-27', 'close_...             False   
  1   [{'close_approach_date': '1909-08-21', 'close_...             False   
  2   [{'close_approach_date': '1910-01-04', 'close_...             False   
  3   [{'close_approach_date': '1910-02-25', 'close_...             False   
  4   [{'close_approach_date': '1900-03-08', 'close_...             False   
...                                                 ...               ...   
8 15  [{'close_approach_date': '1906-03-06', 'close_...             False   
  16  [{'close_approach_date': '1905-08-25', 'close_...             False   
  17  [{'close_approach_date': '1907-10-29', 'close_...             False   
  18  [{'close_approach_date': '1908-04-11', 'close_...             False   
  19  [{'close_approach_date': '1921-07-08', 'close_...             False   

      ... orbital_data.perihelion_distance  orbital_data.perihelion_argument  \
0 0   ...                1.133475820116491                 178.8971619902451   
  1   ...                1.194898892415675                 156.2275765643465   
  2   ...                1.062256387810841                 350.4932417946612   
  3   ...                1.244627432512162                 132.4281002671606   
  4   ...                 1.08246658685257                 26.64392069638918   
...   ...                              ...                               ...   
8 15  ...      

In [56]:
df_concatenado.to_csv("near_earth_objects-january_february_1995-raw.csv")